In [63]:
import requests as rq
import string
from bs4 import BeautifulSoup
import pandas as pd
import time

In [ ]:
#Cabecera navegador
cabecera = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36"}

#Url sitio scraping
url_base = "https://www.amazon.es"
url_base_dp = url_base + "/dp/"

#Página inicial con el resultado de la búsqueda de libros de fisica
url_inicial="https://www.amazon.es/s/ref=lp_902503031_nr_n_4?fst=as%3Aoff&rh=n%3A599364031%2Cn%3A%21599365031%2Cn%3A902503031%2Cn%3A902508031&bbn=902503031&ie=UTF8&qid=1571919270&rnid=902503031"

#Abrimos sesion
sesion=rq.Session()
sesion.post(url_base, headers=cabecera)

#Accedemos a las página inicial
pagina = sesion.get(url_inicial, headers=cabecera)
soup = BeautifulSoup(pagina.content)
num_pagina = 1

#Obtenemos número máximo de páginas de resultados
texto_nmax =soup.find(id="pagn")
contador_max = texto_nmax.find_all('span')
pagina_max = int(contador_max[7].get_text())


#Creamos dataframe libros_df
libros_df = pd.DataFrame(columns=('titulo', 'precio', 'sinopsis', 'formato', 'editor','coleccion', 'idioma', 'isbn'))

#Función que extrae los atributos del libro que nos interesan
def datos_libro(arg):
    #Retardamos peticiones
    t0 = time.time()
    libro = sesion.get(arg, headers=cabecera)
    response_delay = time.time() - t0
    time.sleep(10 * response_delay)
    #Accedemos al contenido de la página
    soup_libro = BeautifulSoup(libro.content)
    reg_libro = []
    #Accedemos al titulo
    reg_libro.append(soup_libro.title.get_text().strip())
    #Accedemos al precio
    precio_libro =  soup_libro.find(id="buyNewSection")
    if precio_libro == None:
        reg_libro.append("")
    else:
        reg_libro.append(precio_libro.get_text().strip('\n'))
    #Accedemos a la sinopsis
    sinopsis=soup_libro.find(id="bookDescription_feature_div")
    if sinopsis == None:
        reg_libro.append("")
    else:
        reg_libro.append(sinopsis.div)
    #Accedemos detalle libro
    detalle_libro=soup_libro.find(id="detail_bullets_id")
    if detalle_libro == None:
        reg_libro.append(["", "", "", "", ""])
    else:
        detalle = detalle_libro.find_all('li')
        formato = detalle[0].get_text().strip()
        reg_libro.append(formato)
        editor = detalle[1].get_text().strip()
        reg_libro.append(editor)
        coleccion=detalle[2].get_text().strip()
        reg_libro.append(coleccion)
        idioma = detalle[3].get_text().strip()
        reg_libro.append(idioma)
        isbn = detalle[5].get_text().strip()
        reg_libro.append(isbn)
    return reg_libro
        

#Recorremos las paginas resultados
for paginas in range(pagina_max):

    if num_pagina == 1:
        resultado = soup.find(id="mainResults")
        for tag_li in resultado.find_all('li'):
            asin = tag_li.get('data-asin')
            #Edicion física
            if asin[0] != 'B':
                url = url_base_dp+asin+"/"
                libro_amazon = datos_libro(url)
                libros_df.loc[len(libros_df)]=libro_amazon
                
        url_pag_sig = soup.find(id="pagnNextLink")
        url_pag_sig = url_base + url_pag_sig.get("href")
        num_pagina = num_pagina + 1
    else:
        pagina = sesion.get(url_pag_sig, headers=cabecera)
        soup = BeautifulSoup(pagina.content)
        tag_div = soup.find_all('div')
        for item in tag_div:
            if item.has_attr('data-asin'):
                asin = item.get('data-asin')
                #Edicion física
                if asin[0] != 'B':
                    url = url_base_dp+asin+"/"
                    libro_amazon = datos_libro(url)
                    libros_df.loc[len(libros_df)]=libro_amazon
        if num_pagina < pagina_max:
            partes_url = url_pag_sig.split("&")
            url_0 = partes_url[0]
            pagina = 'page='+ str(num_pagina+1)
            url_2 = partes_url[2]
            ref = 'ref=lp_902508031_pg_' + str(num_pagina)
            url_pag_sig = url_0 + '&' + pagina + '&' + url_2 +'&' + ref
            num_pagina = num_pagina + 1

In [ ]:
libros_df.to_csv('librosFisicaAmazon.csv')